In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import pyodbc

In [86]:
def create_connection_to_netezza(dbname="xxx", 
                                     dbhost="xx.xx.xx.xx", 
                                     dbport="xxxx",
                                     dbuser="xxxxx",
                                     dbpass="xxxxxx"
                                     ):
    
        connection_string = "Driver={};servername={}"\
                              ";port={};database={}"\
                              ";username={};password={};".format(
                                  "{NetezzaSQL}",dbhost, dbport, 
                                  dbname, dbuser, dbpass
                              )
                              
        conn_engine = pyodbc.connect(connection_string)
        db_cursor = conn_engine.cursor()
        return (conn_engine, db_cursor)

In [3]:
def reduce_by_key(data, group_key, sort_key):
    data_dedup = data.sort_values(group_key + sort_key, ascending=False).drop_duplicates(group_key, keep='first')
    for key in group_key:
        data_dedup = data_dedup[data_dedup[key]==data_dedup[key]]
    return data_dedup

def format_data_with_schema(data, schema):
    new_data = pd.DataFrame()
    for (old, new) in schema:
        if not(new==new) or len(new)==0 or (new is None):
            new = old
        new_data[new] = data[old]
    return new_data

In [4]:
komtraxk_data_schema = [
    ("manufacturer", "manufacturer"),
    ("model", "unit_model"),
    ("type", "model_type"),
    ("serial", "unit_serial_number"),
    ("comm_setting_group", "comm_setting_group"),
    ("cgid", "comm_setting_group_id"),
    ("td", "comm_setting_group_td"),
    ("db_group", "db_group"),
    ("did", "db_group_id"),
    ("country", "db_group_country"),
    ("customer", "customer"),
    ("cid", "customer_id"),
    ("cust_code", "customer_code"),
    ("country2", "customer_country"),
    ("datetime", "gps_datetime"),
    ("origin", "gps_origin"),
    ("datum", "gps_datum"),
    ("latitude", "gps_latitude"),
    ("longitude", "gps_longitude"),
    ("local_date", "smr_local_date"),
    ("smr_value", "smr_value"),
    ("local_date3", "daily_gauge_local_date"),
    ("last_fuel", "daily_gauge_fuel_last"),
    ("full", "daily_gauge_fuel_full"),
    ("thresh", "daily_gauge_fuel_thres"),
    ("max_water_temp", "daily_gauge_max_water_temp"),
    ("full4", "daily_gauge_water_temp_full"),
    ("thresh5", "daily_gauge_water_temp_thres"),
    ("local_date6", "daily_smr_local_date"),
    ("daily_value", "daily_smr_value"),
    ("daily_map", "daily_smr_map"),
    ("local_date7", "daily_fuel_local_date"),
    ("daily_value8", "daily_fuel_value"),
    ("local_date9", "daily_actual_work_local_date"),
    ("daily_value10", "daily_actual_work_value"),
    ("local_date11", "daily_travel_local_date"),
    ("daily_value12", "daily_travel_value"),
    ("local_date13", "daily_opmode_local_date"),
    ("daily_emode", "daily_emode"),
    ("daily_pmode", "daily_pmode"),
    ("daily_bmode", "daily_bmode"),
    ("daily_lmode", "daily_lmode"),
    ("daily_llmode", "daily_llmode"),
    ("daily_lmmode", "daily_lmmode"),
    ("daily_lhmode", "daily_lhmode"),
    ("daily_mlmode", "daily_mlmode"),
    ("daily_mmmode", "daily_mmmode"),
    ("daily_mhmode", "daily_mhmode"),
    ("daily_hlmode", "daily_hlmode"),
    ("daily_hmhmode", "daily_hmhmode"),
    ("local_date14", "daily_caution_local_date"),
    ("caution", "daily_caution_caution_name"),
    ("caid", "daily_caution_caution_id"),
    ("mainte_id", "maintenance_id"),
    ("mainte_desc", "maintenance_desc"),
    ("remain_time", "maintenance_remain_time"),
    ("latest_maintenance_datetime", "maintenance_latest_datetime"),
    ("latest_maintenance_smr", "maintenance_latest_smr"),
    ("lifetime", "maintenance_lifetime"),
    ("interval", "maintenance_interval"),
    ("thresh15", "maintenance_threshold"),
    ("available_flag", "maintenance_available_flag"),
    ("local_date16", "abnormality_local_date"),
    ("code", "abnormality_code"),
    ("name17", "abnormality_name"),
    ("datetime18", "abnormality_datetime"),
    ("smr_value19", "abnormality_smr"),
    ("onmonitor", "abnomrmality_onmonitor"),
    ("omid", "abnormality_onmonitor_id"),
    ("action_level", "abnormality_action_level"),
    ("type20", "abnormality_type"),
    ("atid", "abnormality_type_id"),
    ("local_date21", "daily_att_local_date"),
    ("daily_value22", "daily_att_value")
]

In [5]:
daily_attachment_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_att_local_date", "local_date"),
    ("daily_att_value", ""),
]

daily_abnormality_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("abnormality_code",""),
    ("abnormality_name",""),
    ("abnormality_datetime","local_datetime"),
    ("abnormality_smr",""),
    ("abnomrmality_onmonitor",""),
    ("abnormality_onmonitor_id",""),
    ("abnormality_action_level",""),
    ("abnormality_type",""),
    ("abnormality_type_id","")
]

maintenance_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("maintenance_id",""),
    ("maintenance_desc",""),
    ("maintenance_remain_time",""),
    ("maintenance_latest_datetime","local_date"),
    ("maintenance_latest_smr",""),
    ("maintenance_lifetime",""),
    ("maintenance_interval",""),
    ("maintenance_threshold",""),
    ("maintenance_available_flag","")
]

daily_caution_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_caution_local_date","local_date"),
    ("daily_caution_caution_name",""),
    ("daily_caution_caution_id","")
]

daily_opmode_local_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_opmode_local_date","local_date"),
    ("daily_emode", ""),
    ("daily_pmode",""),
    ("daily_bmode",""),
    ("daily_lmode",""),
    ("daily_llmode",""),
    ("daily_lmmode",""),
    ("daily_lhmode",""),
    ("daily_mlmode",""),
    ("daily_mmmode",""),
    ("daily_mhmode",""),
    ("daily_hlmode",""),
    ("daily_hmhmode","")
]

daily_travel_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_travel_local_date","local_date"),
    ("daily_travel_value",""),
]

daily_actual_work_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_actual_work_local_date","local_date"),
    ("daily_actual_work_value",""),
]

daily_fuel_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_fuel_local_date","local_date"),
    ("daily_fuel_value",""),
]

daily_smr_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_smr_local_date","local_date"),
    ("daily_smr_value",""),
    ("daily_smr_map",""),
]

daily_gauge_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("daily_gauge_local_date","local_date"),
    ("daily_gauge_fuel_last",""),
    ("daily_gauge_max_water_temp",""),
]

machine_smr_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("smr_local_date","local_date"),
    ("smr_value","value")
]

machine_identification_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number", ""),
    ('gps_datetime',""),
    ('gps_latitude',''), 
    ('gps_longitude',''),
    ("comm_setting_group", ""),
    ("comm_setting_group_id", ""),
    ("comm_setting_group_td", ""),
    ("db_group", ""),
    ("db_group_id", ""),
    ("db_group_country", ""),
    ("customer", ""),
    ("customer_id", ""),
    ("customer_code", ""),
    ("customer_country", ""),
]

machine_gps_schema = [
    ("manufacturer", ""),
    ("unit_model", ""),
    ("model_type", ""),
    ("unit_serial_number",""),
    ('gps_datetime',""),
    ('gps_origin', ''),
    ('gps_datum',''), 
    ('gps_latitude',''), 
    ('gps_longitude',''),
]

In [6]:
komtrax_xml_data = pd.read_excel("Komtrax XML New Format 110119.xlsx")
print(len(komtrax_xml_data))
komtrax_xml_data.head(5)

193417


,name,value,total_results,manufacturer,model,type,serial,comm_setting_group,cgid,td,...,name17,datetime18,smr_value19,onmonitor,omid,action_level,type20,atid,local_date21,daily_value22
0,kba,KAP,28952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,key,C156836990FA9B4,28952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,db_registered,true,28952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,country,ID,28952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,NaN,NaN,28952,KOMATSU,D155A,6R,85016,Cr Indonesia 8,W0026,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [7]:
komtrax_xml_data_fmt = format_data_with_schema(komtrax_xml_data, schema=komtraxk_data_schema)
komtrax_xml_data_fmt = komtrax_xml_data_fmt[komtrax_xml_data_fmt['unit_serial_number'].notnull()]
komtrax_xml_data_fmt.columns

Index(['manufacturer', 'unit_model', 'model_type', 'unit_serial_number',
       'comm_setting_group', 'comm_setting_group_id', 'comm_setting_group_td',
       'db_group', 'db_group_id', 'db_group_country', 'customer',
       'customer_id', 'customer_code', 'customer_country', 'gps_datetime',
       'gps_origin', 'gps_datum', 'gps_latitude', 'gps_longitude',
       'smr_local_date', 'smr_value', 'daily_gauge_local_date',
       'daily_gauge_fuel_last', 'daily_gauge_fuel_full',
       'daily_gauge_fuel_thres', 'daily_gauge_max_water_temp',
       'daily_gauge_water_temp_full', 'daily_gauge_water_temp_thres',
       'daily_smr_local_date', 'daily_smr_value', 'daily_smr_map',
       'daily_fuel_local_date', 'daily_fuel_value',
       'daily_actual_work_local_date', 'daily_actual_work_value',
       'daily_travel_local_date', 'daily_travel_value',
       'daily_opmode_local_date', 'daily_emode', 'daily_pmode', 'daily_bmode',
       'daily_lmode', 'daily_llmode', 'daily_lmmode', 'daily_lhmod

In [8]:
group_key=['unit_model', 'model_type', 'unit_serial_number', 'local_date']
sort_key=['gps_datetime']

In [51]:
machine_gps =reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=machine_gps_schema).dropna(),
    group_key=['unit_model', 'model_type', 'unit_serial_number','gps_datetime'], sort_key=['gps_latitude'])
machine_gps.to_csv('machine_gps.csv', index=False)

machine_identification = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=machine_identification_schema).dropna(),
    group_key=['unit_model', 'model_type', 'unit_serial_number'], sort_key=sort_key)
machine_identification.to_csv('machine_identification.csv', index=False)

machine_smr = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=machine_smr_schema).dropna(),
    group_key, sort_key)
machine_smr.to_csv('machine_smr.csv', index=False)

daily_smr = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_smr_schema).dropna(),
    group_key, sort_key)
daily_smr.to_csv('daily_smr.csv', index=False)

daily_travel = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_travel_schema).dropna(),
    group_key, sort_key)
daily_travel.to_csv('daily_travel.csv', index=False)

daily_attachement = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_attachment_schema).dropna(),
    group_key, sort_key)
daily_attachement.to_csv('daily_attachement.csv', index=False)

daily_opmode_local = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_opmode_local_schema),
    group_key, sort_key)
daily_opmode_local.to_csv('daily_opmode_local.csv', index=False)

daily_gauge = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_gauge_schema).dropna(),
    group_key, sort_key)
daily_gauge.to_csv('daily_gauge.csv', index=False)

daily_fuel = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_fuel_schema).dropna(),
    group_key, sort_key)
daily_fuel.to_csv('daily_fuel.csv', index=False)

daily_caution = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_caution_schema).dropna(),
    group_key=group_key + ['daily_caution_caution_id',], sort_key=sort_key)
daily_caution.to_csv('daily_caution.csv', index=False)

daily_actual_work = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_actual_work_schema).dropna(),
    group_key, sort_key)
daily_actual_work.to_csv('daily_actual_work.csv', index=False)

daily_abnormality = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=daily_abnormality_schema).dropna(),
    group_key=['unit_model', 'model_type', 'unit_serial_number', 'abnormality_code','local_datetime'],
    sort_key=sort_key)
daily_abnormality.to_csv('daily_abnormality.csv', index=False)

maintenance = reduce_by_key(
    format_data_with_schema(komtrax_xml_data_fmt, schema=maintenance_schema).dropna(),
    group_key=group_key + ['maintenance_id',], sort_key=sort_key)
maintenance.to_csv('maintenance.csv', index=False)